# First Consolidated Code for Group - v0.1 2023-11-02

In [ ]:
## Consolidate code from Adrian, Amanda, & Zachary into one document

## Code review together to eliminate redundancies and optimise
## 1. Pulling in data from TSV file
## 2. ETL: Order into DataFrame and arrange columns & calculated columns
## 3. Comment consolidated code
## 4. Make notes on existing charts and determine their value-add to presentation

# Adrian Code

## adrian_starter_code_v01.ipynb

## Airline Analysis - Project 1 - Adrian Santos  
#### Last updated: 2023-11-01 -- v0.1

### Import data file, process loaded in DataFrame, scrub for nulls, recast date variable, & reorder columns

In [ ]:
# Initial set-up - Import libraries
import pandas as pd

In [ ]:
# Load airline route data into DataFrame - define datatype for integers
df = pd.read_table('./Resources/flight_edges.tsv', 
     names=['Origin Airport', 
            'Destination Airport', 
            'Origin City', 
            'Destination City',                                          
            'Passengers', 
            'Seats', 
            'Flights', 
            'Distance', 
            'Fly Date', 
            'Origin Population', 
            'Destination Population',
           ],
     dtype={'Passengers': int, 
            'Seats': int, 
            'Flights': int, 
            'Distance': int, 
            'Fly Date': str, 
            'Origin Population': int, 
            'Destination Population': int
           },
     chunksize=1000000
                  )

In [ ]:
# Recast 'Fly Date' from YYYYMM format to separate columns of 'Month' and 'Year'
def flydate_recast(date:str):
    date = list(date)
    monthList = date[-2:]
    month = ''
    month = month.join(monthList)
    month = str(month)
    yearList = date[:4]
    year = ''
    year = year.join(yearList)
    year = str(year)
    return month, year

In [ ]:
# Create an empty list to store the chunks of 1,000,000 rows at a time
chunks = []

In [ ]:
# Iterate over each chunk of data
for chunk in df:
    
    # Drop rows with null values
    chunk.dropna(inplace=True)
    
    # Recast 'Fly Date' from YYYYMM format to columns for 'Month' and 'Year'
    chunk[['Month', 'Year']] = chunk['Fly Date'].apply(lambda x: pd.Series(flydate_recast(x)))
    
    # Append the modified chunk to the list
    chunks.append(chunk)

In [ ]:
# Concatenate all of the chunks into one DataFrame
df = pd.concat(chunks)

In [ ]:
# Return the number of rows and columns in the DataFrame
df.shape

In [ ]:
# Reorder column order so 'Month' and 'Year' are repositioned to come after 'Fly Date'
column_reorder = ['Origin Airport', 
             'Destination Airport', 
             'Origin City', 
             'Destination City', 
             'Passengers', 
             'Seats', 
             'Flights', 
             'Distance', 
             'Fly Date', 
             'Month', 
             'Year', 
             'Origin Population', 
             'Destination Population'
            ]
df = df.reindex(columns=column_reorder)

In [ ]:
# Return updated DataFrame
df.head()

## Scrubbed DataFrame in place --  Begin preliminary analyses

In [ ]:
# Step 01 - Perform exploratory analysis by aggregating on various attributes

In [ ]:
# NOTES: 

# Amanda Code

## Top 10 Routes.ipynb

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_table('flight_edges.tsv', names=['Origin Airport', 'Destination Airport', 'Origin City', 'Destination City', 
                                                          'Passengers', 'Seats', 'Flights', 'Distance', 'Fly Date', 'Origin Population', 'Destination Population'])
df.head()

In [ ]:
df["Route"] = df["Origin Airport"] + "_" + df["Destination Airport"]
passenger_count = df[["Route", "Passengers"]].groupby("Route").sum().sort_values("Passengers", ascending=False)

demand = passenger_count.merge(df[["Route", "Origin Airport", "Origin City", "Destination Airport", "Destination City"]], on="Route", how="inner").groupby("Route").first().sort_values("Passengers", ascending=False)
demand.head(10)

In [ ]:
# NOTES: 

# Zachary Code

## Data Cleanup.ipynb

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
df = pd.read_table('./Resources/flight_edges.tsv', names=['Origin Airport', 'Destination Airport', 'Origin City', 'Destination City', 
                                                          'Passangers', 'Seats', 'Flights', 'Distance', 'Fly Date', 'Origin Population', 'Destination Population'])
df.head()

In [ ]:
df.shape[0]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape[0]

In [ ]:
len(df['Origin Airport'].unique())

In [ ]:
#DATE IS IN YYYYMM FORMAT. NEED TO SPLIT INTO COLUMNS
def splitYearMonth (date:int):
    date = str(date)
    date = list(date)
    yearList = date[:4]
    year = ''
    year = year.join(yearList)
    year = int(year)
    monthList = date[-2:]
    month = ''
    month = month.join(monthList)
    month = int(month)
    return year, month

In [ ]:
df['Fly Date'] = df['Fly Date'].astype(str)
df['Year'], df['Month'] = df['Fly Date'].str[:4], df['Fly Date'].str[4:]
df.head()

In [ ]:
df.to_csv('../08-EV-charging/Resources/8minuteMayhem', index=False)

In [ ]:
df2 = pd.read_csv('./Resources/8minuteMayhem')

In [ ]:
df2['Percent Full'] = round((df2['Passangers'] / df2['Seats']), 2)
df2.head()

In [ ]:
df2 = df2[['Origin Airport', 'Origin City', 'Origin Population', 'Destination Airport',  'Destination City', 'Destination Population','Distance', 'Flights', 'Passangers', 
		   'Seats', 'Percent Full', 'Fly Date', 'Month', 'Year']]
df2.head()

In [ ]:
df2.to_csv('./Resources/bigCleanData.csv', index=False)

In [ ]:
# Notes:

## Chart Generation.ipynb

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels.api as sm

df = pd.read_csv('./Resources/bigCleanData.csv')
df.head()

In [ ]:
# Remove where two columns are equal, this step may be skipped
# New question: how may airports have flights originating and arriving at themselves?
df = df[df['Destination Airport'] != df['Origin Airport']]

In [ ]:
#Count of unique routes, summing the value in the Flights column (number of flights in the given month)
uniqueRoutes = df.groupby(['Origin Airport', 'Destination Airport'])['Flights'].sum().reset_index()
uniqueRoutes.head()

In [ ]:
#Total number of unique routes
uniqueRoutes.shape[0]

### Pie Chart With Dest Code

In [ ]:
#Get count of each occurance of a destination code
destFlights = df.groupby(['Destination Airport'])['Flights'].sum().reset_index()
destFlights.sort_values('Flights', inplace=True, ascending=False)
destFlights.head()

In [ ]:
numEntries = 25
shortened = destFlights.iloc[:numEntries]

plt.pie(shortened['Flights'], labels=shortened['Destination Airport'], autopct='%1.1f%%', radius=2)
plt.title("Percent of flights terminating at a given airport", y=1.4)
plt.show()

shortSum = shortened['Flights'].sum()
longSum = destFlights['Flights'].sum()
pctShown = round(((shortSum/longSum) * 100), 2)

print(f"The above chart accounts for {pctShown}% of flights")

### Pie Chart With Orig Code

In [ ]:
origFlights = df.groupby(['Origin Airport'])['Flights'].sum().reset_index()
origFlights.sort_values('Flights', inplace=True, ascending=False)
origFlights.head()

In [ ]:
numEntries = 25
shortened = origFlights.iloc[:numEntries]

plt.pie(shortened['Flights'], labels=shortened['Origin Airport'], autopct='%1.1f%%', radius=2)
plt.title("Percent of flights starting at a given airport", y=1.4)
plt.show()

shortSum = shortened['Flights'].sum()
longSum = origFlights['Flights'].sum()
pctShown = round(((shortSum/longSum) * 100), 2)

print(f"The above chart accounts for {pctShown}% of flights")

### Percent Full vs Route Frequency

In [ ]:
avgPctFull = df.groupby(['Origin Airport', 'Destination Airport'])['Percent Full'].mean().reset_index()
avgPctFull.head()

In [ ]:
fullFrame = pd.merge(uniqueRoutes, avgPctFull, on=['Origin Airport', 'Destination Airport'])
withNA = len(fullFrame)
fullFrame.dropna(inplace=True)
droppedNA = len(fullFrame)

print(f"There where {withNA - droppedNA} flights with NA values, assumed to be empty flights")

In [ ]:
test = df[df['Percent Full'] > 1]
print(f'There are {len(test)} instances of flights running over capacity')

### Chart X01: Percent Full vs Number of Flights

In [ ]:
plt.scatter(fullFrame['Flights'], fullFrame['Percent Full'])
plt.xlabel('Number of Flights')
plt.ylabel('Average Percent Full for each route')
plt.title('Percent Full vs Number of Flights')
plt.show()

### Chart X02: Distribution -- needs explanation by Zachary

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(fullFrame['Flights'], fullFrame['Percent Full'])
print(pvalue)

print(f'p = {slope}f + {intercept}'

In [ ]:
plt.hist(fullFrame[['Percent Full']], bins = 30)
plt.show()

### Route Frequency vs Destination Population (E)

#### Shelved for now, running into odd issues with getting the population of the destination location

In [ ]:
df2 = df.copy()
oriDest = 'Destination'

portCode = f'{oriDest} Airport'
coi = f'{oriDest} Population'

codeLookup = df2[[portCode, coi]].drop_duplicates(keep='first')
codeLookup = codeLookup.groupby(portCode)[coi].mean().reset_index()
codeLookup

In [ ]:
#Column of interest will be abreviated to coi
dataOfInterest = pd.merge(uniqueRoutes, codeLookup, on=portCode)
dataOfInterest.tail()

In [ ]:
withNA = len(dataOfInterest)
dataOfInterest.dropna(inplace=True)
droppedNA = len(dataOfInterest)

#No NA values found, skipping print statement
#print(f"There where {withNA - droppedNA} flights with NA values, assumed to be empty flights")

In [ ]:
#dataOfInterest[coi].max()
dataOfInterest.sort_values(coi, ascending=False, inplace=True)
dataOfInterest

### Chart X03: Destination Population vs Number of Flights

In [ ]:
plt.scatter(dataOfInterest[coi], dataOfInterest['Flights'])
plt.ylabel('Number of Flights')
plt.xlabel(coi)
plt.title(f'{coi} vs Number of Flights')
plt.show()

### Number of Passangers Flown per Year and Month

### Big City Small City Correlation

In [ ]:
# NOTES: 